In [12]:
import numpy as np
import pandas as pd
import scipy.stats as st
import os
from convert import *
import datetime
import time
import cvxpy as cp
from matplotlib import pyplot as plt
import matplotlib
from sklearn.decomposition import PCA
import sklearn.covariance as skcov
import seaborn as sns

In [13]:
hourly_df = pd.read_csv(r'D:\Capstone_Data\cleaned\hourly_data_bid_ask_foreign.csv')
timestamps = [datetime.datetime.strptime(item[:-6], '%Y-%m-%d %H') for item in hourly_df.time_utc]
symbols = np.unique([item[:12] for item in hourly_df.columns[1:]]).astype(str)

mid_price_mat = np.zeros((len(timestamps), len(symbols)))
bid_price_mat = np.zeros((len(timestamps), len(symbols)))
ask_price_mat = np.zeros((len(timestamps), len(symbols)))

#calculating mid prices
for i in range(len(symbols)):
    mid_price_mat[:, i] = (hourly_df[symbols[i]+'_bid'] + hourly_df[symbols[i]+'_ask'])/2
    bid_price_mat[:, i] = hourly_df[symbols[i]+'_bid']
    ask_price_mat[:, i] = hourly_df[symbols[i]+'_ask']
    
#calculating transaction cost percentage for buying or selling
C = (ask_price_mat - mid_price_mat)/mid_price_mat

#use mid price to get the returns
rets = mid_price_mat[1:]/mid_price_mat[:-1] - 1

#splitting the returns and timestamps for the training and test set
train_rets = rets[:-20*23]
test_rets = rets[-20*23:]

train_ts = timestamps[:-20*23]
test_ts = timestamps[-20*23:]

#transaction cost for the training set
train_C = C[1:-20*23]

In [14]:
def pnl_stats(pnl, pnl_len):
    '''
    calculate annualized statistics
    '''
    
    pnl = pnl[-pnl_len:]
    pnl = pnl/pnl[0]
    #standard deviation
    rets = pnl[1:]/pnl[:-1] - 1
    std = np.std(rets) * np.sqrt(23*252)
    
    #total return on investment
    roi = np.power(pnl[-1], 23*252/len(pnl))-1
    
    #sharpe ratio 
    sr = roi/std
    
    return roi, std, sr
    
def sample_cov(rets):
    return np.cov(rets.T, ddof=0)

def shrunk_cov(rets, alpha):
    return skcov.ShrunkCovariance(shrinkage=alpha).fit(train_rets).covariance_

def LW_cov(rets):
    return skcov.LedoitWolf().fit(rets).covariance_

# def online_cov(rets, lb_window, alpha):
#     n_iter = len(rets)-lb_window+1
#     res = np.zeros((n_iter, 10, 10))
#     for i in range(n_iter):
#         lb_rets = rets[i:i+lb_window]
#         if i == 0:
#             res[i] = sample_cov(lb_rets)
#         else:
#             res[i] = (1-alpha)*sample_cov(lb_rets) + alpha*sample_cov(lb_rets)
#     return res
    
def backtest(train_ts, train_rets, reb_freq, lb_window, cov_type, alpha=None, a_ol = None):
    '''
    Input:
    train_ts: timestamps for plotting
    train_rets: array of historical returns for backtesting
    reb_freq: rebalancing frequency/holding period
    lb_window: lookback window size
    cov_type: types of covariance matrix
    constrained: whether has the non-negativity constraint
    
    Output:
    '''
    time_plot = train_ts[lb_window:] #timestamps for plotting
    n_iters = int((train_rets.shape[0] - lb_window)/reb_freq)+1
    pnl = np.array([1]) #initialize portfolio pnl
    all_weights = np.zeros((1,10)) #initialize portfoio weights to all 0
    
    #calculate the online covariance matrices if necessary
        
    error_percent = 0
    for i in range(n_iters):
        weights_prev = all_weights[-1]
        #calculate mu and Sigma
        lb_rets = train_rets[i*reb_freq : lb_window + i*reb_freq,:]
        mu = np.mean(lb_rets, axis=0)
        if cov_type == 'sample':
            Sigma = sample_cov(lb_rets)
        elif cov_type == 'shrunk':
            Sigma = shrunk_cov(lb_rets, alpha)
        elif cov_type == 'LedoitWolf':
            Sigma = LW_cov(lb_rets)
        elif cov_type == 'online':
            if i == 0:
                Sigma = sample_cov(lb_rets)
            else:
                Sigma = (1-a_ol)*sample_cov(lb_rets) + a_ol * Sigma
        
        #C_curr = train_C[lb_window+i*reb_freq-1] #the transaction cost matrix
        C_curr = np.zeros((10,))
        
        #performing Markowitz
        
        if np.any(Sigma==0):
            weights = all_weights[-1]
        else:
            Sigma_ = 1/np.min(np.abs(Sigma)) * Sigma
            w = cp.Variable(10)
            risk = cp.quad_form(w, Sigma_)
            ret = mu.T @ w
            trans_cost = cp.abs(w-weights_prev) @ C_curr
            kappa = 1
            obj = risk
            prob = cp.Problem(cp.Minimize(obj), [cp.sum(w) == 1, w>=0])
            try:
                prob.solve(solver='OSQP')
                weights = w.value
            except:
                try:
                    prob.solve(solver='ECOS')
                    weights = w.value
                except:
                    try:
                        prob.solve(solver='ECOS_BB')
                        weights = w.value
                    except:
                        try:
                            prob.solve(solver='SCS')
                            weights = w.value
                        except:
                            weights = all_weights[-1]
                            error_percent += 1/n_iters
        
        all_weights[-1] = weights
        
        #net the transaction cost
        pnl_post_reb = pnl[-1] - np.abs(weights-weights_prev)*pnl[-1] @ C_curr #pnl after rebalancing
        pnl[-1] = pnl_post_reb #updating
        
        #calculating weights and pnl in the validation set
        valid_rets = train_rets[lb_window + i*reb_freq : lb_window+(i+1)*reb_freq,:]
        valid_pnl = pnl_post_reb*weights*np.cumprod(1+valid_rets, axis=0) #the pnl in each currency
        total_pnl = np.sum(valid_pnl, axis=1)
        valid_weights = valid_pnl/np.repeat(total_pnl,10).reshape(valid_pnl.shape)
        
        #updating the weights & pnl in the valid set
        all_weights = np.vstack((all_weights, valid_weights))
        pnl = np.hstack((pnl, total_pnl))
    
    #plotting the pnl and save
    plt.plot(time_plot, pnl, label='M.V. Port')
    plt.xticks(rotation=45)
    plt.legend()
    if cov_type == 'shrunk':
        plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\{}\pnl\LB{} HP{}.png'.format(cov_type, alpha, lb_window, reb_freq))
    elif cov_type == 'online':
        plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\{}\pnl\LB{} HP{}.png'.format(cov_type, a_ol, lb_window, reb_freq))
    else:
        plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\pnl\LB{} HP{}.png'.format(cov_type, lb_window, reb_freq))
    plt.clf()
    plt.close()
    #plotting the portfolio weights
    plt.figure(figsize=(8,4))
    plt.stackplot(time_plot, all_weights.T, labels = [symbol[:3] for symbol in symbols])
    plt.xticks(rotation=45)
    plt.legend()
    if cov_type == 'shrunk':
        plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\{}\weights\LB{} HP{}.png'.format(cov_type, alpha, lb_window, reb_freq))
    elif cov_type == 'online':
        plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\{}\weights\LB{} HP{}.png'.format(cov_type, a_ol, lb_window, reb_freq))
    else:
        plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\weights\LB{} HP{}.png'.format(cov_type, lb_window, reb_freq))
    plt.clf()
    plt.close()
    return all_weights, pnl, error_percent

In [15]:
#defining the set of parameters
lb_windows = np.arange(20,250,10)
reb_freqs = np.arange(1,21,1)

standard_pnl_len = len(train_ts) - max(lb_windows)

xdata_hm = [] 
ydata_hm = [] #for plotting

#getting the x & y axis ready for plotting
for lb_window in lb_windows:
    for reb_freq in reb_freqs:
        xdata_hm.append(lb_window)
        ydata_hm.append(reb_freq)

In [25]:
#running backtest for sample covariance
cov_type = 'sample'
roi_stats = []
std_stats = []
sr_stats = []
error_stats = []
configs = []

for lb_window in lb_windows:
    for reb_freq in reb_freqs:
        configs.append('LB:{} HP:{}'.format(lb_window, reb_freq))
        _, pnl, error_percent = backtest(train_ts, train_rets, reb_freq, lb_window, cov_type)
        roi, std, sr = pnl_stats(pnl, standard_pnl_len)
        roi_stats.append(roi)
        std_stats.append(std)
        sr_stats.append(sr)
        error_stats.append(error_percent)

In [26]:
df_out = pd.DataFrame({'Configurations': configs, 'ROI': roi_stats, 'STD': std_stats, 'SR': sr_stats, 'ERR': error_stats})
df_out.to_csv(r'D:\Capstone_Data\result_hourly_without_mu\{}\summary.csv'.format(cov_type), index=False)
    
#plot the heat map
data_plot = pd.DataFrame({'Lookback Window': xdata_hm, 'Holding Period': ydata_hm, 
                          'ROI': roi_stats, 'STD': std_stats, 'SR': sr_stats, 'ERR': error_stats})
for metric in ['ROI', 'STD', 'SR', 'ERR']:
    if metric == 'STD':
        #ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric), vmin=0.05, vmax=0.1)
        ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric))
    else:
        ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric))
    ax.invert_yaxis()
    ax.set_title(cov_type+metric)
    plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\heatmaps\{}.png'.format(cov_type, metric))
    plt.clf()
    plt.close()


In [ ]:
#running backtest for shrunk covariance matrix
cov_type = 'shrunk'
alphas = np.round(np.arange(0.1,0.6,0.1),1)
for alpha in alphas:
    roi_stats = []
    std_stats = []
    sr_stats = []
    error_stats = []
    configs = []
    for lb_window in lb_windows:
        for reb_freq in reb_freqs:
            configs.append('LB:{} HP:{}'.format(lb_window, reb_freq))
            _, pnl, error_percent = backtest(train_ts, train_rets, reb_freq, lb_window, cov_type, alpha)
            roi, std, sr = pnl_stats(pnl, standard_pnl_len)
            roi_stats.append(roi)
            std_stats.append(std)
            sr_stats.append(sr)
            error_stats.append(error_percent)

    #save the summary statisitics in a csv file
    df_out = pd.DataFrame({'Configurations': configs, 'ROI': roi_stats, 'STD': std_stats, 'SR': sr_stats, 'ERR': error_stats})
    df_out.to_csv(r'D:\Capstone_Data\result_hourly_without_mu\{}\{}\summary.csv'.format(cov_type, alpha), index=False)

    #plot the heat map
    data_plot = pd.DataFrame({'Lookback Window': xdata_hm, 'Holding Period': ydata_hm, 
                              'ROI': roi_stats, 'STD': std_stats, 'SR': sr_stats, 'ERR': error_stats})
    for metric in ['ROI', 'STD', 'SR', 'ERR']:
        if metric == 'STD':
            ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric), vmin=0.05, vmax=0.1)
        else:
            ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric))
        ax.invert_yaxis()
        ax.set_title(cov_type+metric)
        plt.savefig(r'D:\Capstone_Data\result_daily\{}\{}\heatmaps\{}.png'.format(cov_type, alpha, metric))
        plt.clf()
        plt.close()

In [32]:
#running backtest for LedoitWolf covariance
cov_type = 'LedoitWolf'
roi_stats = []
std_stats = []
sr_stats = []
error_stats = []
configs = []

for lb_window in lb_windows:
    for reb_freq in reb_freqs:
        configs.append('LB:{} HP:{}'.format(lb_window, reb_freq))
        _, pnl, error_percent = backtest(train_ts, train_rets, reb_freq, lb_window, cov_type)
        roi, std, sr = pnl_stats(pnl, standard_pnl_len)
        roi_stats.append(roi)
        std_stats.append(std)
        sr_stats.append(sr)
        error_stats.append(error_percent)

In [33]:
#save the summary statisitics in a csv file
df_out = pd.DataFrame({'Configurations': configs, 'ROI': roi_stats, 'STD': std_stats, 'SR': sr_stats, 'ERR': error_stats})
df_out.to_csv(r'D:\Capstone_Data\result_hourly_without_mu\{}\summary.csv'.format(cov_type), index=False)
    
#plot the heat map
data_plot = pd.DataFrame({'Lookback Window': xdata_hm, 'Holding Period': ydata_hm, 
                          'ROI': roi_stats, 'STD': std_stats, 'SR': sr_stats, 'ERR': error_stats})
for metric in ['ROI', 'STD', 'SR', 'ERR']:
    if metric == 'STD':
        #ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric), vmin=0.05, vmax=0.1)
        ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric))
    else:
        ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric))
    ax.invert_yaxis()
    ax.set_title(cov_type+metric)
    plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\heatmaps\{}.png'.format(cov_type, metric))
    plt.clf()
    plt.close()


In [16]:
#running backtest for online covariance
cov_type = 'online'
a_ol = 0.01
roi_stats = []
std_stats = []
sr_stats = []
error_stats = []
configs = []

for lb_window in lb_windows:
    for reb_freq in reb_freqs:
        configs.append('LB:{} HP:{}'.format(lb_window, reb_freq))
        _, pnl, error_percent = backtest(train_ts, train_rets, reb_freq, lb_window, cov_type, a_ol=a_ol)
        roi, std, sr = pnl_stats(pnl, standard_pnl_len)
        roi_stats.append(roi)
        std_stats.append(std)
        sr_stats.append(sr)
        error_stats.append(error_percent)
        
#save the summary statisitics in a csv file
df_out = pd.DataFrame({'Configurations': configs, 'ROI': roi_stats, 'STD': std_stats, 'SR': sr_stats, 'ERR': error_stats})
df_out.to_csv(r'D:\Capstone_Data\result_hourly_without_mu\{}\{}\summary.csv'.format(cov_type, a_ol), index=False)
    
#plot the heat map
data_plot = pd.DataFrame({'Lookback Window': xdata_hm, 'Holding Period': ydata_hm, 
                          'ROI': roi_stats, 'STD': std_stats, 'SR': sr_stats, 'ERR': error_stats})
for metric in ['ROI', 'STD', 'SR', 'ERR']:
    if metric == 'STD':
        #ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric), vmin=0.05, vmax=0.1)
        ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric))
    else:
        ax = sns.heatmap(data_plot.pivot('Lookback Window', 'Holding Period', metric))
    ax.invert_yaxis()
    ax.set_title(cov_type+metric)
    plt.savefig(r'D:\Capstone_Data\result_hourly_without_mu\{}\{}\heatmaps\{}.png'.format(cov_type, a_ol, metric))
    plt.clf()
    plt.close()